## ResNet을 활용하여 모델 구성하기

In [ ]:
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def get_model(num_classes):
    # ImageNet으로 학습된 모델을 불러옵니다.
    # 각 인자에 대한 설명은 5.4절을 참고하세요.
    # inception_v3 = InceptionV3(weights = 'imagenet', include_top = False, input_shape = (32, 32, 3))
    resnet = ResNet50(weights = 'imagenet', include_top = False, input_shape = (32, 32, 3))
    
    # 불러온 모델의 마지막 출력층과 연결합니다. 
    x = GlobalAveragePooling2D()(resnet.output)
    x = Dense(num_classes, activation = 'softmax')(x)
    
    # 모델을 생성합니다.
    model = Model(inputs = resnet.input, outputs = x)
    
    return model

model = get_model(num_classes=10)
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['acc'])

# 모델 구조를 확인해보세요!
# 또는 plot_model()로 그려보아도 좋습니다.
# model.summary()

## CIFAR-10 학습시켜보기

In [ ]:
from tensorflow.keras.datasets import cifar10
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 평균과 표준편차는 채널별로 구해줍니다.
x_mean = np.mean(x_train, axis = (0, 1, 2))
x_std = np.std(x_train, axis = (0, 1, 2))

x_train = (x_train - x_mean) / x_std
x_test = (x_test - x_mean) / x_std

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, random_state = 777)

print(x_train.shape, len(y_train))
print(x_val.shape, len(y_val))

# 학습을 진행합니다.
history = model.fit(x_train, y_train,
                    epochs = 30,
                    batch_size = 32,
                    validation_data = (x_val, y_val))